***GENERATED CODE FOR predictingtotalfarebasedonbookingmethodandairline PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PNR', 'transformation_label': 'String Indexer'}], 'feature': 'PNR', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1DD6CU', 'max': 'ZYQIKL', 'missing': '0', 'distinct': '443'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PNR'}, {'feature_label': 'PNR', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PNR')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ticket_Num', 'transformation_label': 'String Indexer'}], 'feature': 'Ticket_Num', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '38657072613.68', 'stddev': '203479034023.5', 'min': '1.25297E+12', 'max': 'YCCK4K', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ticket_Num'}, {'feature_label': 'Ticket_Num', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ticket_Num')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InvoiceDate', 'transformation_label': 'String Indexer'}], 'feature': 'InvoiceDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2019', 'max': '1/9/2019', 'missing': '0', 'distinct': '27'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InvoiceDate'}, {'feature_label': 'InvoiceDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InvoiceDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartDate', 'transformation_label': 'String Indexer'}], 'feature': 'DepartDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/10/2019', 'max': '9/20/2018', 'missing': '0', 'distinct': '85'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartDate'}, {'feature_label': 'DepartDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReturnDate', 'transformation_label': 'String Indexer'}], 'feature': 'ReturnDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/10/2019', 'max': '9/22/2018', 'missing': '0', 'distinct': '97'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReturnDate'}, {'feature_label': 'ReturnDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReturnDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Itinerary', 'transformation_label': 'String Indexer'}], 'feature': 'Itinerary', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '#NAME?', 'max': 'YYZ-YWG-YYZ', 'missing': '0', 'distinct': '378'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Itinerary'}, {'feature_label': 'Itinerary', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Itinerary')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Class', 'transformation_label': 'String Indexer'}], 'feature': 'Class', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'BUSINESS', 'max': 'PREMIUM COACH', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Class'}, {'feature_label': 'Class', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Class')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartAirportCode', 'transformation_label': 'String Indexer'}], 'feature': 'DepartAirportCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' ', 'max': 'YYZ', 'missing': '0', 'distinct': '67'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartAirportCode'}, {'feature_label': 'DepartAirportCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartAirportCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartCity', 'transformation_label': 'String Indexer'}], 'feature': 'DepartCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ADELAIDE, SA', 'max': 'ZHENGZHOU, CHINA         ', 'missing': '0', 'distinct': '140'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartCity'}, {'feature_label': 'DepartCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartCountry', 'transformation_label': 'String Indexer'}], 'feature': 'DepartCountry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AUSTRALIA', 'max': 'VIETNAM', 'missing': '0', 'distinct': '26'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartCountry'}, {'feature_label': 'DepartCountry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartCountry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DestinationAirportCode', 'transformation_label': 'String Indexer'}], 'feature': 'DestinationAirportCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ABQ', 'max': 'YWG', 'missing': '0', 'distinct': '69'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DestinationAirportCode'}, {'feature_label': 'DestinationAirportCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DestinationAirportCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DestinationCity', 'transformation_label': 'String Indexer'}], 'feature': 'DestinationCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ADDIS ABABA (BOLE INTL APT)', 'max': 'SAO PAOLO, BRAZIL        ', 'missing': '0', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DestinationCity'}, {'feature_label': 'DestinationCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DestinationCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DestinationCountry', 'transformation_label': 'String Indexer'}], 'feature': 'DestinationCountry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AUSTRALIA', 'max': 'UNITED STATES', 'missing': '0', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DestinationCountry'}, {'feature_label': 'DestinationCountry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DestinationCountry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternationalDomestic', 'transformation_label': 'String Indexer'}], 'feature': 'InternationalDomestic', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Domestic', 'max': 'Transborder', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternationalDomestic'}, {'feature_label': 'InternationalDomestic', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternationalDomestic')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AirlineCode', 'transformation_label': 'String Indexer'}], 'feature': 'AirlineCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '3K', 'max': 'ZZ', 'missing': '0', 'distinct': '48'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AirlineCode'}, {'feature_label': 'AirlineCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AirlineCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AirlineName', 'transformation_label': 'String Indexer'}], 'feature': 'AirlineName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ADVANCED AIR', 'max': 'WESTJET AIRLINES', 'missing': '0', 'distinct': '49'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AirlineName'}, {'feature_label': 'AirlineName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AirlineName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BookCountry', 'transformation_label': 'String Indexer'}], 'feature': 'BookCountry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Canada', 'max': 'United States', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BookCountry'}, {'feature_label': 'BookCountry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BookCountry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BookMethod', 'transformation_label': 'String Indexer'}], 'feature': 'BookMethod', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Offline', 'max': 'Online', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BookMethod'}, {'feature_label': 'BookMethod', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BookMethod')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '_c27', 'transformation_label': 'String Indexer'}], 'feature': '_c27', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '_c27'}, {'feature_label': '_c27', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('_c27')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run predictingtotalfarebasedonbookingmethodandairlineHooks.ipynb
try:
	#sourcePreExecutionHook()

	totalfarebasedonbookingmethodandairline = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Total Fare based on Booking Method and Airline.csv', 'filename': 'Total Fare based on Booking Method and Airline.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(totalfarebasedonbookingmethodandairline)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run predictingtotalfarebasedonbookingmethodandairlineHooks.ipynb
try:
	#transformationPreExecutionHook()

	predictingtotalfarebasedonbookingmethodandairlineautofe = TransformationMain.run(totalfarebasedonbookingmethodandairline,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "InvoiceNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1235402305.34", "stddev": "588700200.68", "min": "3753314", "max": "2076787875", "missing": "0"}, "updatedLabel": "InvoiceNumber"}, {"transformationsData": [{"feature_label": "PNR", "transformation_label": "String Indexer"}], "feature": "PNR", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1DD6CU", "max": "ZYQIKL", "missing": "0", "distinct": "443"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PNR"}, {"transformationsData": [{"feature_label": "Ticket_Num", "transformation_label": "String Indexer"}], "feature": "Ticket_Num", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "38657072613.68", "stddev": "203479034023.5", "min": "1.25297E+12", "max": "YCCK4K", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ticket_Num"}, {"transformationsData": [{"feature_label": "InvoiceDate", "transformation_label": "String Indexer"}], "feature": "InvoiceDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2019", "max": "1/9/2019", "missing": "0", "distinct": "27"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InvoiceDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PassengerName", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1055793138.7", "stddev": "622974721.46", "min": "659898", "max": "2140155340", "missing": "0"}, "updatedLabel": "PassengerName"}, {"transformationsData": [{"feature_label": "DepartDate", "transformation_label": "String Indexer"}], "feature": "DepartDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/10/2019", "max": "9/20/2018", "missing": "0", "distinct": "85"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartDate"}, {"transformationsData": [{"feature_label": "ReturnDate", "transformation_label": "String Indexer"}], "feature": "ReturnDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/10/2019", "max": "9/22/2018", "missing": "0", "distinct": "97"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReturnDate"}, {"transformationsData": [{"feature_label": "Itinerary", "transformation_label": "String Indexer"}], "feature": "Itinerary", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "#NAME?", "max": "YYZ-YWG-YYZ", "missing": "0", "distinct": "378"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Itinerary"}, {"transformationsData": [{"feature_label": "Class", "transformation_label": "String Indexer"}], "feature": "Class", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "BUSINESS", "max": "PREMIUM COACH", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Client", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1186193733.77", "stddev": "655543433.55", "min": "17363", "max": "2082844838", "missing": "0"}, "updatedLabel": "Client"}, {"transformationsData": [{"feature_label": "DepartAirportCode", "transformation_label": "String Indexer"}], "feature": "DepartAirportCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " ", "max": "YYZ", "missing": "0", "distinct": "67"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartAirportCode"}, {"transformationsData": [{"feature_label": "DepartCity", "transformation_label": "String Indexer"}], "feature": "DepartCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ADELAIDE, SA", "max": "ZHENGZHOU, CHINA         ", "missing": "0", "distinct": "140"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartCity"}, {"transformationsData": [{"feature_label": "DepartCountry", "transformation_label": "String Indexer"}], "feature": "DepartCountry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AUSTRALIA", "max": "VIETNAM", "missing": "0", "distinct": "26"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartCountry"}, {"transformationsData": [{"feature_label": "DestinationAirportCode", "transformation_label": "String Indexer"}], "feature": "DestinationAirportCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ABQ", "max": "YWG", "missing": "0", "distinct": "69"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DestinationAirportCode"}, {"transformationsData": [{"feature_label": "DestinationCity", "transformation_label": "String Indexer"}], "feature": "DestinationCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ADDIS ABABA (BOLE INTL APT)", "max": "SAO PAOLO, BRAZIL        ", "missing": "0", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DestinationCity"}, {"transformationsData": [{"feature_label": "DestinationCountry", "transformation_label": "String Indexer"}], "feature": "DestinationCountry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AUSTRALIA", "max": "UNITED STATES", "missing": "0", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DestinationCountry"}, {"transformationsData": [{"feature_label": "InternationalDomestic", "transformation_label": "String Indexer"}], "feature": "InternationalDomestic", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Domestic", "max": "Transborder", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternationalDomestic"}, {"transformationsData": [{"feature_label": "AirlineCode", "transformation_label": "String Indexer"}], "feature": "AirlineCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "3K", "max": "ZZ", "missing": "0", "distinct": "48"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AirlineCode"}, {"transformationsData": [{"feature_label": "AirlineName", "transformation_label": "String Indexer"}], "feature": "AirlineName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ADVANCED AIR", "max": "WESTJET AIRLINES", "missing": "0", "distinct": "49"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AirlineName"}, {"transformationsData": [{"feature_label": "BookCountry", "transformation_label": "String Indexer"}], "feature": "BookCountry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Canada", "max": "United States", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BookCountry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalFare", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "900.62", "stddev": "1633.23", "min": "0.0", "max": "11440.43", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalFare"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Savings", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "86.5", "stddev": "557.04", "min": "-2100.03", "max": "8513.23", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Savings"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LostSavings", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "27.65", "stddev": "152.56", "min": "0.0", "max": "2878.77", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LostSavings"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AdvanceDays", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "23.47", "stddev": "25.43", "min": "0", "max": "178", "missing": "0"}, "updatedLabel": "AdvanceDays"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TicketCount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "TicketCount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MilesFLown", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3155.45", "stddev": "4212.31", "min": "-9009", "max": "21770", "missing": "0"}, "updatedLabel": "MilesFLown"}, {"transformationsData": [{"feature_label": "BookMethod", "transformation_label": "String Indexer"}], "feature": "BookMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Offline", "max": "Online", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BookMethod"}, {"transformationsData": [{"feature_label": "_c27", "transformation_label": "String Indexer"}], "feature": "_c27", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "_c27"}]}))

	#transformationPostExecutionHook(predictingtotalfarebasedonbookingmethodandairlineautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run predictingtotalfarebasedonbookingmethodandairlineHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(predictingtotalfarebasedonbookingmethodandairlineautofe, ["InvoiceNumber", "PassengerName", "Client", "TotalFare", "Savings", "LostSavings", "AdvanceDays", "TicketCount", "MilesFLown", "PNR_stringindexer", "Ticket_Num_stringindexer", "InvoiceDate_stringindexer", "DepartDate_stringindexer", "ReturnDate_stringindexer", "Itinerary_stringindexer", "Class_stringindexer", "DepartAirportCode_stringindexer", "DepartCity_stringindexer", "DepartCountry_stringindexer", "DestinationAirportCode_stringindexer", "DestinationCity_stringindexer", "DestinationCountry_stringindexer", "InternationalDomestic_stringindexer", "AirlineCode_stringindexer", "AirlineName_stringindexer", "BookCountry_stringindexer", "_c27_stringindexer"], "BookMethod_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

